## Linear Regression

* Build the linear regression model based on the dataset and features from data preprocessing secton
* Evlauate LR model with following metrics
>* R^2 score
>* Mean Square Error (MSE)
>* Mean Absolute Eroor (MAE)
* Visualise prediction vs actual values
* Performance and Parameter tuning specific for linear regression
>* Gradient Descent and Cost Function
>* Visualise data by scatter plot to further evaluate features (independent variables) which are appropriate for linear regeression
>* Rebuild model based on the new findings
>* Evaluate and compare the performance

In [ ]:
%% capture
%run data_prep_and_analysis.ipynb

In [ ]:
# Evaluate Models
def evaluate_model(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, mse, r2

In [ ]:
# create function to include repeated steps for building model, evaluating model, and visualising the results
def build_lr_model(X, y, model_name):

    # build the model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    lm = LinearRegression(fit_intercept=True)
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)

    # evaluate the model
    print('X_train shape :', X_train.shape)
    print('X_test shape :',X_test.shape)
    print('y_train shape :',y_train.shape)
    print('y_test shape :',y_test.shape)
    print("")
    
    # Model performance metrics
    mae, mse, r2score = evaluate_model(y_test, y_pred)
    print("Model Evaluation")
    print("Linear Regression Metrics (MAE, MSE, R^2):")
    print(mae, mse, r2score)

    print("")
    # Coefficient
    print('Coefficients:')
    print(lm.coef_)

    print("")
    # Intercept
    print('Intercept:')
    print(lm.intercept_)

    # Plot predictions vs. actual values
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
    plt.title('Predictions vs. Actual Values (Linear Regression) for {}'.format(model_name))
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.show()

    return { 'Model': model_name, 'MAE': mae, 'MSE': mse, 'R^2': r2score }
     

### Build model on selected features

In [ ]:
# Create an empty list to store results
lr_performance = []

In [ ]:
print('Selected features:', selected_features)

In [ ]:
# Train and Evaluate Linear Regression
result = build_lr_model(X_train, y_train, "Dataset using selected features")
lr_performance.append(result)

### Performance and Parameter Tuning for Linear Regression

In [ ]:
# Train and evaluate Linear Regression with Gradient Descent
gdlr = SGDRegressor(loss='squared_error', penalty='l1', learning_rate='optimal', n_iter_no_change=10)
gdlr.fit(X_train, y_train)
y_pred_gd = gdlr.predict(X_test)

In [ ]:
# Apply an ensemble method with K-Fold Cross Validation to the data 
ensemble = SGDRegressor(loss='squared_error', penalty='l1', learning_rate='optimal', n_iter_no_change=10)
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
mae = cross_val_score(ensemble, X_train, y_train, scoring = "neg_mean_absolute_error", cv = kf, n_jobs=4) # model runs in 4 parallel threads
print(f'Ensemble MAE evaluation table: {mae}')
mse = cross_val_score(ensemble, X_train, y_train, scoring = "neg_mean_squared_error", cv = kf, n_jobs=4) # model runs in 4 parallel threads
print(f'Ensemble MSE evaluation table: {mse}')
r2 = cross_val_score(ensemble, X_train, y_train, scoring = "r2", cv = kf, n_jobs=4) # model runs in 4 parallel threads
print(f'Ensemble r2 evaluation table {r2}')

### Visualise relationship between selected features
In this section, the selected features used in the previous section will be tuned to compare the performance

>['Emission Value', 'VKM', 'AADT', 'Length (m)', 'VehicleType', 'Toid', 'Interaction_Length_Speed']

In [ ]:
# Investigate all the elements within each Feature 

for column in df:
    unique_vals = np.unique(df[column])
    nr_values = len(unique_vals)
    if nr_values < 20: # show unqiue if the total is less than 20
        print('The number of values for feature {} :{} -- {}'.format(column, nr_values,unique_vals))
    else:
        print('The number of values for feature {} :{}'.format(column, nr_values))

### Visualise the selected features relationship for each location

In [ ]:
# Visualise the data using seaborn Pariplots
# Since it is very time consuming on large datasets, divide the dataset into subset
# Location_ExactCut is a good candidate for filtering as it has 4 unique values = df['Location_ExactCut'].unique()
unique_loc = df['Location_ExactCut'].unique()
unique_loc = sorted(unique_loc)
location_dataset = []
for loc in unique_loc:
    df_loc = df[df['Location_ExactCut'] == loc]
    location_dataset.append(df_loc)

In [ ]:
# visualise the selected features relationship for each location 
hue_column = 'VehicleType'

In [ ]:
# location 0 - Central
df_loc = location_dataset[0]
g = sns.pairplot(df_loc[selected_features], hue = hue_column, height = 5)

In [ ]:
# location 1 - External
df_loc = location_dataset[1]
g = sns.pairplot(df_loc[selected_features], hue = hue_column, height = 5)

In [ ]:
# location 2 - Inner
df_loc = location_dataset[2]
g = sns.pairplot(df_loc[selected_features], hue = hue_column, height = 5)

In [ ]:
# location 3 - External
df_loc = location_dataset[3]
g = sns.pairplot(df_loc[selected_features], hue = hue_column, height = 5)

### Notes
* Based on the result of the pairplot of the selected features, strong linear relationship between VKM and Emission Value is observed while AADT and Emission value do not indicate linear relationship
* Additionally, the linear relationship between VKM and Emission Value is even stronger for individual vehicle type

### Plot VKM and Emission Value for individual vehicle type for Location 0 - Central

In [ ]:
# Visualise the data and regression model fits across a FacetGrid for individual vehicle type in one location
df_loc = location_dataset[0]
g = sns.lmplot(x = 'VKM', y = 'Emission Value', data = df_loc, col = 'VehicleType', col_wrap = 3, height = 5, 
              scatter_kws = {'color':'green'}, ci = False)

### Plot VKM and Emission Value for individual vehicle type for Location 1 - External

In [ ]:
# Visualise the data and regression model fits across a FacetGrid for individual vehicle type in one location
df_loc = location_dataset[1]
g = sns.lmplot(x = 'VKM', y = 'Emission Value', data = df_loc, col = 'VehicleType', col_wrap = 3, height = 5, 
              scatter_kws = {'color':'green'}, ci = False)

### Plot VKM and Emission Value for individual vehicle type for Location 2 - Inner

In [ ]:
# Visualise the data and regression model fits across a FacetGrid for individual vehicle type in one location
df_loc = location_dataset[2]
g = sns.lmplot(x = 'VKM', y = 'Emission Value', data = df_loc, col = 'VehicleType', col_wrap = 3, height = 5, 
              scatter_kws = {'color':'green'}, ci = False)


### Plot VKM and Emission Value for individual vehicle type for Location 3 - Outer

In [ ]:
# Visualise the data and regression model fits across a FacetGrid for individual vehicle type in one location
df_loc = location_dataset[3]
g = sns.lmplot(x = 'VKM', y = 'Emission Value', data = df_loc, col = 'VehicleType', col_wrap = 3, height = 5, 
              scatter_kws = {'color':'green'}, ci = False)

#### Analysis result

* Based on the result of lmplot between VKM and Emission for individual location,
>* It shows the same observation as pairplot with all features where strong linear relationship can be found between VKM and Emission Value
>* VKM and Emission value for individual vehicle type show the similar trend in all locations

#### Rebuild model with new findings
* Based on the visualisation analayis, the result indicates a good linear relationship between VKM and Emission Value for individual vehicle type
* Try to train the linear regression model by dataset of individual vehicle type and evalulate the performance

In [ ]:
# List to store performance metric of individual vehicle type
vt_performance = []

# Retrieve the unique vehicle type
unique_vt = df['VehicleType'].unique()
unique_vt = sorted(unique_vt)
print("Unique vehicle type:", unique_vt)


In [ ]:
# run the model for individual vehicle type
for vt in unique_vt:
    df_vt = df[df['VehicleType'] == vt]
    X_vt = df_vt[['VKM']]
    y_vt = df_vt['Emission Value']

    result = build_lr_model(X_vt, y_vt, 'Dataset using feature VKM and vehicle type {}'.format(vt_mapping[vt]))
    lr_performance.append(result)
    vt_performance.append(result)

### Get average performance from the result of individual vehicle type

In [ ]:
df_vt_performance = pd.DataFrame(vt_performance)

mean_mae = df_vt_performance['MAE'].mean()
mean_mse = df_vt_performance['MSE'].mean()
mean_r2 = df_vt_performance['R^2'].mean()

print("Average performance metrics by using VKM for individual vehicle type:", mean_mae, mean_mse, mean_r2)
     

In [ ]:
# try to put back the vehicle type as selected features to see the performance
new_features = ['VKM', 'VehicleType']
X_t2 = df[new_features]
y_t2 = df['Emission Value']

result =  build_lr_model(X_t2, y_t2, 'Dataset using features VKM and VehicleType')
lr_performance.append(result)
     

In [ ]:
# Print the performance comparison of different datasets
lr_metrics_overview = pd.DataFrame(lr_performance)
lr_metrics_overview
     

### Conclusion
* The original model's performance was suboptimal because scatter plots revealed no linear relationship between most features and the Emission Value.
* Rebuilding the model using VKM and Emission Value, which show a strong linear relationship, significantly improved performance.
* Linear regression may not be the optimal solution for this dataset with the initially selected features.
* To effectively use linear regression for this dataset, separate models should be built for each vehicle type, and their individual results should be combined for the final prediction.